In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import json
import re 
from pandas.io.json import json_normalize

In [2]:
match_df = pd.read_pickle('~/Desktop/lol_data/match_data.pickle')
#winner_df =  pd.read_pickle('~/Desktop/lol_data/match_winner_data.pickle')
#loser_df = pd.read_pickle('~/Desktop/lol_data/match_loser_data.pickle')

# Filter Ranked Games
- Only Ranked games (gameMode = Classic)
- remove columns not needed
- get information on the participants into the df

In [3]:
match_df = match_df[match_df["gameMode"]=="CLASSIC"]

In [4]:
match_df = (match_df.drop(columns=["status.message","status.status_code",
                                   "participantIdentities","gameMode", 
                                   "gameType","platformId", "seasonId", 
                                   "mapId","queueId"]))

In [10]:
sample = match_df.iloc[:10000][:]

In [11]:
sample.reset_index(drop=True, inplace=True)

In [12]:
sample.to_pickle('~/Desktop/lol_data/match_01.pickle')

In [35]:
sample = pd.read_pickle('~/Desktop/lol_data/match_01.pickle')

In [36]:
def parse_participant(participant_dict):
    keys_to_keep = ["participantId",'teamId','championId','spell1Id','spell2Id']
    stats_keys = (['win',"item0", "item1","item2","item3","item4","item5","item6", "kills", "deaths", "assists",
                  "doubleKills", "tripleKills", "quadraKills", "pentaKills", "goldEarned","role", "lane"])
    e = {}
    for key, value in participant_dict.items():
        if key in keys_to_keep:
            e[key] = value
        elif type(value) == dict:
            for key2, value2 in value.items():
                if key2 in stats_keys:
                    e[key2] = value2
                    
    return e

In [37]:
for i in range(len(sample)):
    d = sample.loc[i,"participants"]
    e = [parse_participant(part) for part in d]
    sample["participants"][i] = e

/Users/andreasglassl/opt/miniconda3/envs/backpack_banana/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [38]:
sample.to_pickle('~/Desktop/lol_data/match_01.pickle')

In [48]:
participants_df = (pd
                   .json_normalize(sample["participants"].iloc[0][:])
                   .assign(gameDuration = sample.iloc[0][1]
                           ,gameId=sample.iloc[0][2]
                           ,gameVersion=sample.iloc[0][3]))

In [54]:
for i in range(8000,10000):
    participants_df = (participants_df
                       .append(pd.json_normalize(sample["participants"].iloc[i][:])
                                                                         .assign(gameDuration = sample.iloc[i][1]
                                                                         ,gameId=sample.iloc[i][2]
                                                                         ,gameVersion=sample.iloc[i][3]
                                                                         ,ignore_index=True)))

In [57]:
participants_df.reset_index(drop=True, inplace=True)

In [58]:
participants_df.to_csv('~/Desktop/lol_data/match_01.csv', index=False)

In [59]:
winner_df =  pd.read_pickle('~/Desktop/lol_data/match_winner_data.pickle')
loser_df = pd.read_pickle('~/Desktop/lol_data/match_loser_data.pickle')

In [61]:
winner_df.to_csv('~/Desktop/lol_data/match_winner_data.csv')
loser_df.to_csv('~/Desktop/lol_data/match_loser_data.csv')